In [138]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [139]:
# Display the total number of players
# Make sure there is no duplicate 'SN' rows
player_count = purchase_data.drop_duplicates(subset='SN',keep='first')

# Collect info on the dataframe
# player_count.info()
total = player_count["SN"].count()
total_players = pd.DataFrame({"Total Players":[total]})
total_players
#-----------------------------------------------------
###Alternatively, you can find the unique value counts of the length of the "SN" col
#total_players_2 = len(purchase_data["SN"].value_counts())

,Total Players
0,576


In [140]:
# Run basic calculations to obtain number of unique items, average price, etc.
total_unique_items = purchase_data["Item ID"].nunique()
avg_price = purchase_data["Price"].mean()
total_purchases = purchase_data["Item ID"].count()
total_revenue = purchase_data["Price"].sum()
#total_unique_items
#avg_price
#total_purchases
#total_revenue

# Create a summary data frame to hold the results
summary_data = {"Number of Unique Items": [total_unique_items], "Average Price":[avg_price],
                "Number of Purchases":[total_purchases], "Total Revenue":total_revenue}
summary_df = pd.DataFrame(summary_data)

# Optional: give the displayed data cleaner formatting
#---------------------------------------------------------------------
#summary_df["Average Price"] = summary_df["Average Price"].map("${:.2f}".format)
#summary_df["Total Revenue"] = summary_df["Total Revenue"].map("${:,.2f}".format)
###Alternatively, you can run a for loop to apply formating for each key-value pairing...
format_map={"Average Price":"${:,.2f}", "Total Revenue":"${:,.2f}"}
for key,value in format_map.items():
    summary_df[key] = summary_df[key].apply(value.format)
#----------------------------------------------------------------------

# Display the summary data frame
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [141]:
# Percentage and Count of Male Players
# Percentage and Count of Female Playersto
# Percentage and Count of Other / Non-Disclosed
#------------------------------------------------------
###Different ways to colect player counts by gender...
#gender_count = player_count.groupby("Gender").size()
#gender_count = pd.value_counts(player_count["Gender"])
gender_count = player_count["Gender"].value_counts()
#------------------------------------------------------
percent_gender = gender_count/total*100

# Convert Series into Dictionaries
gender_count_dict = gender_count.to_dict()
#gender_count_dict
percent_gender_dict = percent_gender.to_dict()
#percent_gender_dict

# DataFrame with Dictionaries, transposed, indexed, and formatted...Note: the DF of the counts became Floats when converting from
# Dict to DF...so, format back to (int) is needed...
gender_demographics = pd.DataFrame([gender_count_dict,percent_gender_dict],
                  index = ["Total Count","Percentage of Players"]).transpose()
gender_demographics["Percentage of Players"] = gender_demographics["Percentage of Players"].map("%{:.2f}".format)
gender_demographics["Total Count"] = gender_demographics["Total Count"].astype(int)
gender_demographics.index.name = "Gender"
gender_demographics.sort_values(["Total Count"], ascending = False)
#--------------------------------------------------------
###Alternatively, you can just simply create a DF from the Series and create a Dictionary within
#gender = pd.DataFrame({"#Players":gender_count, "%Players":percent_gender})
#gender


,Total Count,Percentage of Players
Gender,,
Male,484,%84.03
Female,81,%14.06
Other / Non-Disclosed,11,%1.91


In [142]:
# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
# Group DF by gender
grouped_gender_purchase = purchase_data.groupby("Gender")
# Calculations: total count, avg price, total price, avg purchase/person
gender_purchase_count = purchase_data["Gender"].value_counts()
avg_purchase_price = grouped_gender_purchase["Price"].mean()
purchase_total = grouped_gender_purchase["Price"].sum()
avg_purchase_person = purchase_total/gender_count

# Create a summary data frame to hold the results
gender_purchase_analysis = pd.DataFrame({"Purchase Count":gender_purchase_count,
                                        "Average Purchase Price":avg_purchase_price,
                                        "Total Purchase Value":purchase_total,
                                        "Average Total Purchase per Person":avg_purchase_person})

# Optional: give the displayed data cleaner formatting
# Display the summary data frame
gender_purchase_analysis.index.name = "Gender"
gender_purchase_analysis.style.format({"Average Purchase Price":"${:,.2f}",
                                      "Total Purchase Value":"${:,.2f}",
                                      "Average Total Purchase per Person":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [143]:
# Establish bins for ages
age_bins = [0,9,14,19,24,29,34,39,120]
age_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
# Categorize the existing players using the age bins. Hint: use pd.cut()
age_bins_pd = pd.cut(player_count["Age"], age_bins, labels=age_labels)


# Calculate the numbers and percentages by age group
age_counts = age_bins_pd.value_counts()
sorted_age_counts = age_counts.sort_index()
percent_age = sorted_age_counts/total*100

# Create a summary data frame to hold the results
age_demographics = pd.DataFrame({"Total Count":sorted_age_counts,
                                "Percentage of Players":percent_age})

# Format age DF
# Display Age Demographics Table
age_demographics.index.name = "Age Range"
age_demographics.style.format({"Percentage of Players":"%{:.2f}"})


,Total Count,Percentage of Players
Age Range,,
<10,17,%2.95
10-14,22,%3.82
15-19,107,%18.58
20-24,258,%44.79
25-29,77,%13.37
30-34,52,%9.03
35-39,31,%5.38
40+,12,%2.08


In [144]:
# Bin the purchase_data data frame by age
age_bins = [0,9,14,19,24,29,34,39,120]
age_labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

# Create a Series of age ranges and store it into the original DF to be grouped with groupby.
purchase_data["Age Range"] = pd.cut(purchase_data["Age"],age_bins, labels=age_labels)
grouped_age = purchase_data.groupby("Age Range")

# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
purchase_count_age = grouped_age["Purchase ID"].count()
avg_purchase_price_age = grouped_age["Price"].mean()
purchase_total_age = grouped_age["Price"].sum()
avg_purchase_person_age = purchase_total_age/sorted_age_counts

# Create a summary data frame to hold the results
age_purchase_analysis = pd.DataFrame({"Purchase Count":purchase_count_age,
                                     "Average Purchase Price":avg_purchase_price_age, 
                                      "Total Purchase Value":purchase_total_age,
                                     "Average Total Purchases Per Person":avg_purchase_person_age})

# Optional: give the displayed data cleaner formatting
# Display the summary data frame
age_purchase_analysis.index.name = "Age Range"
age_purchase_analysis.style.format({"Average Purchase Price":"${:,.2f}",
                                   "Total Purchase Value":"${:,.2f}",
                                   "Average Total Purchases Per Person":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchases Per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [145]:
# Run basic calculations to obtain the results in the table below
grouped_sn = purchase_data.groupby("SN")
purchase_count_sn = grouped_sn["Purchase ID"].count()
avg_purchase_sn = grouped_sn["Price"].mean()
purchase_total_sn = grouped_sn["Price"].sum()
# Create a summary data frame to hold the results
top_spenders = pd.DataFrame({"Purchase Count":purchase_count_sn,
                            "Average Purchase Price":avg_purchase_sn,
                            "Total Purchase Value":purchase_total_sn})
# Sort the total purchase value column in descending order
sorted_top_spenders = top_spenders.sort_values(["Total Purchase Value"], ascending = False)

# Optional: give the displayed data cleaner formatting
# Display a preview of the summary data frame
sorted_top_spenders.head().style.format({"Average Purchase Price":"${:,.2f}",
                                 "Total Purchase Value":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [146]:
# Retrieve the Item ID, Item Name, and Item Price columns
items = purchase_data[["Item ID", "Item Name", "Price"]]

# Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
grouped_items = items.groupby(["Item ID", "Item Name"])
item_purchase_count = grouped_items["Price"].count()
item_purchase_total = grouped_items["Price"].sum()
item_price = item_purchase_total/item_purchase_count

# Create a summary data frame to hold the results
popular_items = pd.DataFrame({"Purchase Count":item_purchase_count,
                             "Item Price":item_price,
                             "Total Purchase Value":item_purchase_total})
# Sort the purchase count column in descending order
sorted_popular_items_count = popular_items.sort_values(["Purchase Count"], ascending = False)

# Optional: give the displayed data cleaner formatting
# Display a preview of the summary data frame
sorted_popular_items_count.head().style.format({"Item Price":"${:,.2f}",
                                  "Total Purchase Value":"${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [147]:
# Sort the above table by total purchase value in descending order
sorted_popular_items_total = popular_items.sort_values(["Total Purchase Value"], ascending=False)

# Optional: give the displayed data cleaner formatting
# Display a preview of the data frame
sorted_popular_items_total.head().style.format({"Item Price":"${:,.2f}",
                                  "Total Purchase Value":"${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
